In [5]:
import cv2
import easyocr
from PIL import Image
import numpy as np
from src.inference import inference
from collections import deque, Counter
from difflib import SequenceMatcher

# Função para cortar a placa usando as coordenadas da bounding box
def cortar_placa(frame, bounding_box):
    x_min, y_min, x_max, y_max = map(int, bounding_box)
    return frame[y_min:y_max, x_min:x_max]

# Função para melhorar a qualidade da imagem antes do OCR
def preprocess_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast_enhanced = cv2.equalizeHist(gray)
    
    kernel = np.array([[0, -1, 0], 
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(contrast_enhanced, -1, kernel)
    smoothed = cv2.GaussianBlur(sharpened, (3, 3), 0)
    
    return smoothed

# Função para realizar OCR na imagem da placa
def detectar_caracteres(img, leitor):
    img_preprocessada = preprocess_image(img)

    # Definir a lista de caracteres permitidos (apenas letras maiúsculas e números)
    allowlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

    # Fazer OCR na imagem
    resultado = leitor.readtext(np.array(img_preprocessada), allowlist=allowlist)

    # Extrair o texto detectado
    texto_detectado = " ".join([res[1] for res in resultado])

    # Verificar se a placa tem 7 caracteres
    if len(texto_detectado) == 7:
        return texto_detectado
    else:
        return None

# Função para calcular similaridade entre duas strings
def calcular_similaridade(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

# Função para contar caracteres e usar sistema de votação por caractere
def atualizar_votacao_caracteres(placa_atual, nova_placa, votos_caracteres):
    for i, caractere in enumerate(nova_placa):
        if i < len(placa_atual):
            # Adicionar o caractere ao contador de votos na posição i
            votos_caracteres[i].update([caractere])

# Função para formar a placa mais provável com base nos votos de cada caractere
def formar_placa_por_votacao(votos_caracteres):
    return "".join([votos.most_common(1)[0][0] for votos in votos_caracteres])

# Função para detectar se uma nova placa deve ser considerada
def detectar_nova_placa(placa_atual, nova_placa):
    # Contar quantos caracteres são diferentes entre as placas
    diferencas = sum([1 for a, b in zip(placa_atual, nova_placa) if a != b])
    return diferencas >= 4  # Se 4 ou mais caracteres forem diferentes, é uma nova placa

# Função para processar vídeo frame a frame e salvar o resultado
def processar_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erro ao abrir o vídeo: {video_path}")
        return

    # Pegar o tamanho e a taxa de quadros do vídeo original
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Definir o codec e criar o objeto VideoWriter para salvar o vídeo processado
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Inicializar o leitor do EasyOCR fora do loop
    leitor = easyocr.Reader(['pt', 'en'])

    placa_atual = None  # Placa atual inferida
    votos_caracteres = []  # Lista de contadores de votos por posição de caractere

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Chamar o modelo de inferência para obter as coordenadas da placa
        result = inference(frame)
        if len(result.boxes) > 0:  # Verificar se há alguma placa detectada
            bbox = result.boxes.xyxy[0].tolist()  # Obter a primeira caixa delimitadora

            # Cortar a placa com base nas coordenadas da inferência
            placa_img = cortar_placa(frame, bbox)

            # Detectar caracteres da placa
            texto_detectado = detectar_caracteres(placa_img, leitor)
            if texto_detectado is not None:
                if placa_atual is None:
                    # Primeira placa detectada
                    placa_atual = texto_detectado
                    votos_caracteres = [Counter() for _ in range(len(placa_atual))]
                    print(f"Nova placa detectada: {placa_atual}")
                else:
                    if detectar_nova_placa(placa_atual, texto_detectado):
                        # Considerar uma nova placa
                        placa_atual = texto_detectado
                        votos_caracteres = [Counter() for _ in range(len(placa_atual))]
                        print(f"Nova placa detectada: {placa_atual}")
                    else:
                        # Atualizar os votos de cada caractere
                        atualizar_votacao_caracteres(placa_atual, texto_detectado, votos_caracteres)
                        # Formar a nova placa inferida com base nos votos
                        placa_inferida = formar_placa_por_votacao(votos_caracteres)
                        if placa_inferida != placa_atual:
                            placa_atual = placa_inferida
                            print(f"Placa inferida: {placa_atual}")

            # Desenhar o bounding box e o texto detectado no frame
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
            cv2.putText(frame, placa_atual, (int(bbox[0]), int(bbox[1]) - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Escrever o frame processado no arquivo de vídeo de saída
        out.write(frame)

    # Liberar o vídeo e o objeto VideoWriter
    cap.release()
    out.release()
    print(f"Processamento concluído. Vídeo salvo em: {output_path}")

# Caminho para o vídeo de entrada e de saída
video_path = 'organic-data/video_teste_4.mp4'
output_path = 'resultado_igo_vote_2.mp4'

# Processar o vídeo e salvar o resultado
processar_video(video_path, output_path)



0: 640x384 2 objects, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 11.1ms
Speed: 2.1ms preprocess, 11.1ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 14.0ms
Speed: 4.0ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 objects, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
Nova placa detectada: 5LAE628

0: 640x384 2 objects, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 objects, 17.1ms
Speed: 2.0ms preprocess, 17.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
Placa inferida: 5LA8628

0: 640x384 1 object, 8.4ms
Speed: 2.0ms preprocess, 8.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 object, 8.8ms
Speed: 1.0ms preprocess, 8.8ms inf